In [1]:
import json
import uuid
from datetime import datetime
from pathlib import Path
import requests  # 用于请求 Ollama 本地 API

# === CONFIG ===
INPUT_FILE = "memcubes_all.json"
OUTPUT_FILE = "memcube_memory_output_with_embed.json"

# === EMBEDDING via Ollama (nomic-embed-text:latest) ===
def get_embedding(text):
    url = "http://localhost:11434/api/embeddings"  # Ollama 默认端口
    payload = {
        "model": "nomic-embed-text:latest",
        "prompt": text
    }
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()["embedding"]
    except Exception as e:
        print(f"⚠️ 获取 embedding 失败：{e}")
        return None

# === TIME STAMP ===
def iso_now():
    return datetime.now().isoformat()

# === CREATE MEMORY NODE ===
def create_memory_node(content, entities, key, memory_type="LongTermMemory"):
    now = iso_now()
    node_id = str(uuid.uuid4())
    embedding = get_embedding(content)
    return {
        "id": node_id,
        "memory": content,
        "metadata": {
            "user_id": "",
            "session_id": "",
            "status": "activated",
            "type": "fact",
            "confidence": 0.99,
            "entities": entities,
            "tags": ["事件"] if "事件" in key else ["关系"],
            "updated_at": now,
            "memory_type": memory_type,
            "key": key,
            "sources": [],
            "embedding": embedding,
            "created_at": now,
            "usage": [],
            "background": ""
        }
    }



In [3]:

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    memcube_data = json.load(f)

nodes = []
edges = []

for character, data in memcube_data.items():
    previous_event_id = None

    # === EVENTS ===
    for event in data.get("events", []):
        memory_text = f"{character}在{event.get('time')}于{event.get('location')}，因为{event.get('motivation')}，进行了{event.get('action')}，结果是{event.get('impact')}。"
        entities = [character] + event.get("involved_entities", [])
        node = create_memory_node(
            content=memory_text,
            entities=entities,
            key=f"{character}的事件：{event.get('action')}"
        )
        nodes.append(node)

        if previous_event_id:
            edges.append({
                "source": previous_event_id,
                "target": node["id"],
                "type": "FOLLOWS"
            })
        previous_event_id = node["id"]

    # === AGGREGATED RELATIONS ===
    relations_texts = []
    seen = set()
    for relation in data.get("relations", []):
        name = relation.get("name") or relation.get("人物") or relation.get("character")
        relation_text = relation.get("relation") or relation.get("relationship") or relation.get("关系")
        if not name or not relation_text:
            continue
        dedup_key = (str(name), str(relation_text))
        if dedup_key in seen:
            continue
        seen.add(dedup_key)
        relations_texts.append(f"与{name}是{relation_text}")

    if relations_texts:
        memory_text = f"{character}" + "，".join(relations_texts) + "。"
        entities = [character] 
        node = create_memory_node(
            content=memory_text,
            entities=entities,
            key=f"{character}的关系汇总",
        )
        nodes.append(node)

# === SAVE TO FILE ===
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump({
        "nodes": nodes,
        "edges": edges
    }, f, ensure_ascii=False, indent=2)

print(f"✅ 完成转换，共生成 {len(nodes)} 个 memory 节点，{len(edges)} 条边")
print(f"📁 输出文件: {OUTPUT_FILE}")



✅ 完成转换，共生成 7684 个 memory 节点，7198 条边
📁 输出文件: memcube_memory_output_with_embed.json
